In [ ]:
import matplotlib.pyplot as plt
import mpl_lego as mplego
import numpy as np
import pandas as pd
import pickle
import tensorflow as tf
import transformers

from pyprojroot import here
from hate_target import keys
from hate_target import utils

from sklearn.metrics import roc_auc_score, average_precision_score
from mpl_lego.labels import bold_text

from hate_measure.nn.classifiers import TargetIdentityClassifier
from hate_target import datasets

%matplotlib inline

In [ ]:
data_path = "~/data/hatespeech/unfiltered_ratings.feather"
rater_quality_path = "~/data/hatespeech/rater_quality_check.csv"
gab_path = "~/data/hatespeech/gab/gab_hate_corpus.tsv"

In [ ]:
gab, gab_target = datasets.load_gab_hate_corpus(
    path=gab_path,
    merge_idl_pol=True,
    fillna=False,
    threshold=0.5)

In [ ]:
data, x, y_hard, y_soft = datasets.load_measuring_hate_groups(data_path, rater_quality_path=rater_quality_path)

In [ ]:
p_train = [y.mean() for y in y_hard]

In [ ]:
model_builder = TargetIdentityClassifier.build_model

In [ ]:
model_kwargs = {
    'transformer': 'roberta-base',
    'max_length': 247,
    'n_dense': 256,
    'dropout_rate': 0.05,
    'pooling': 'mean',
    'mask_pool': False
}

In [ ]:
model = model_builder(**model_kwargs)

In [ ]:
model.save_weights('test.h5')

In [ ]:
model.load_weights('test.h5')

In [ ]:
model.load_weights(here('experiments/exp06a_model.h5'))